# Using influxdb_client library

In [134]:
from influxdb_client import InfluxDBClient, Point, BucketRetentionRules
from datetime import datetime

In [90]:

url = 'http://localhost:8086'
token = '2Wc71suOxmK_X1iQ8pjJd2RAwrn_6azqdQ9vbcW6cUJ3zQv33SPDHjoiqbrmV0VxSACCS9t2_rGql9CxAbca_g=='
org = 'leonardo'
bucket = 'python_test'

In [139]:
client = InfluxDBClient(url=url, token=token, org=org)

In [140]:
client.health()

{'checks': [],
 'commit': '9dcf880fe0',
 'message': 'ready for queries and writes',
 'name': 'influxdb',
 'status': 'pass',
 'version': '2.6.1'}

In [93]:
write_api = client.write_api()
query_api = client.query_api()

In [136]:
# data_to_insert = Point("h2o_level").tag("location", "coyote_creek").field("water_level",12).field("my_field", 8)
data_to_insert = {
    "measurement": "h2o_level",
    "tags": {"location": "coyote_creek"},
    "fields": {"water_level": 8, "my_field":11},
    "time":datetime.now()
}

In [137]:
write_result = write_api.write(bucket=bucket,org=org,record=data_to_insert)


In [138]:
query = 'from(bucket: "python_test")\
|> range(start: -12h)'
result = query_api.query_data_frame(org=org, query=query)
# results = []
# for table in result:
#     for record in table.records:
#         results.append((record.get_value(), record.get_field()))
# results
result


/Users/leonardoleite/.local/share/virtualenvs/fastAPI-NhpKiiZ7/lib/python3.10/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "python_test")|> range(start: -12h) |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


,result,table,_start,_stop,_time,_value,_field,_measurement,location
0,_result,0,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 14:55:52.783831+00:00,10,my_field,h2o_level,coyote_creek
1,_result,0,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 15:00:29.179106+00:00,9,my_field,h2o_level,coyote_creek
2,_result,0,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 16:16:06.873886+00:00,9,my_field,h2o_level,coyote_creek
3,_result,0,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 16:53:25.112366+00:00,8,my_field,h2o_level,coyote_creek
4,_result,0,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 16:57:37.638057+00:00,11,my_field,h2o_level,coyote_creek
5,_result,1,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 13:04:27.716174+00:00,1,water_level,h2o_level,coyote_creek
6,_result,1,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 13:04:41.780537+00:00,1,water_level,h2o_level,coyote_creek
7,_result,1,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 13:05:06.889822+00:00,1,water_level,h2o_level,coyote_creek
8,_result,1,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 13:07:53.794935+00:00,1,water_level,h2o_level,coyote_creek
9,_result,1,2023-01-10 04:58:12.653341+00:00,2023-01-10 16:58:12.653341+00:00,2023-01-10 13:10:54.753481+00:00,1,water_level,h2o_level,coyote_creek


In [130]:
result.describe

<bound method NDFrame.describe of      result  table  ... _measurement      location
0   _result      0  ...    h2o_level  coyote_creek
1   _result      0  ...    h2o_level  coyote_creek
2   _result      0  ...    h2o_level  coyote_creek
3   _result      1  ...    h2o_level  coyote_creek
4   _result      1  ...    h2o_level  coyote_creek
5   _result      1  ...    h2o_level  coyote_creek
6   _result      1  ...    h2o_level  coyote_creek
7   _result      1  ...    h2o_level  coyote_creek
8   _result      1  ...    h2o_level  coyote_creek
9   _result      1  ...    h2o_level  coyote_creek
10  _result      1  ...    h2o_level  coyote_creek
11  _result      1  ...    h2o_level  coyote_creek
12  _result      1  ...    h2o_level  coyote_creek
13  _result      1  ...    h2o_level  coyote_creek

[14 rows x 9 columns]>

In [86]:
retention_rules = BucketRetentionRules(type="expire", every_seconds=3600)
client.buckets_api().create_bucket(bucket_name="created_on_python",retention_rules=retention_rules,org=org)

{'created_at': datetime.datetime(2023, 1, 10, 16, 12, 37, 502280, tzinfo=tzutc()),
 'description': None,
 'id': '4feab0fa347f562e',
 'labels': [],
 'links': {'_self': '/api/v2/buckets/4feab0fa347f562e',
           'labels': '/api/v2/buckets/4feab0fa347f562e/labels',
           'members': '/api/v2/buckets/4feab0fa347f562e/members',
           'org': '/api/v2/orgs/26b2c811b80b549e',
           'owners': '/api/v2/buckets/4feab0fa347f562e/owners',
           'write': '/api/v2/write?org=26b2c811b80b549e&bucket=4feab0fa347f562e'},
 'name': 'created_on_python',
 'org_id': '26b2c811b80b549e',
 'retention_rules': [{'every_seconds': 3600,
                      'shard_group_duration_seconds': 3600,
                      'type': 'expire'}],
 'rp': None,
 'schema_type': None,
 'type': 'user',
 'updated_at': datetime.datetime(2023, 1, 10, 16, 12, 37, 502280, tzinfo=tzutc())}

In [87]:
client.buckets_api().find_buckets()

{'buckets': [{'created_at': datetime.datetime(2023, 1, 10, 16, 12, 37, 502280, tzinfo=tzutc()),
              'description': None,
              'id': '4feab0fa347f562e',
              'labels': [],
              'links': {'_self': '/api/v2/buckets/4feab0fa347f562e',
                        'labels': '/api/v2/buckets/4feab0fa347f562e/labels',
                        'members': '/api/v2/buckets/4feab0fa347f562e/members',
                        'org': '/api/v2/orgs/26b2c811b80b549e',
                        'owners': '/api/v2/buckets/4feab0fa347f562e/owners',
                        'write': '/api/v2/write?org=26b2c811b80b549e&bucket=4feab0fa347f562e'},
              'name': 'created_on_python',
              'org_id': '26b2c811b80b549e',
              'retention_rules': [{'every_seconds': 3600,
                                   'shard_group_duration_seconds': 3600,
                                   'type': 'expire'}],
              'rp': None,
              'schema_type': None,
     

# Using influxdb library

In [103]:
from influxdb import InfluxDBClient as InfluxDBClient2

In [104]:
cl = InfluxDBClient2(host="localhost",port=8086,database="python_test")


In [105]:
cl.create_database("python_db")

InfluxDBClientError: 401: {"code":"unauthorized","message":"Unauthorized"}

In [ ]:
cl.get_list_database()


In [ ]:
cl.switch_database("python_db")